In [51]:
import pandas as pd
import geopandas as gpd
import altair as alt

from pathlib import Path

cwd = Path.cwd()

In [2]:
all_cap = pd.read_parquet(cwd / "all" / "capacity.parquet")
all_cap.head()

,resource_name,zone,tech_type,model,planning_year,case,unit,start_value,end_value,start_MWh,end_MWh,existing,new_build,agg_zone
312737,BASN_batteries_1,BASN,Battery,GenX,2027,20-week-foresight,MW,936.0,936.0,NaN,NaN,True,False,WECC
312738,BASN_conventional_hydroelectric_1,BASN,Hydro,GenX,2027,20-week-foresight,MW,2490.0,2490.0,NaN,NaN,True,False,WECC
312739,BASN_conventional_steam_coal_1,BASN,Coal,GenX,2027,20-week-foresight,MW,1178.0,1178.0,NaN,NaN,True,False,WECC
312740,BASN_conventional_steam_coal_2,BASN,Coal,GenX,2027,20-week-foresight,MW,1822.0,1822.0,NaN,NaN,True,False,WECC
312741,BASN_geothermal_1,BASN,Geothermal,GenX,2027,20-week-foresight,MW,462.0,462.0,NaN,NaN,True,False,WECC


In [29]:
all_cap_range = (
    all_cap.query("case=='full-base-200' and new_build == True")
    .groupby(["model", "planning_year", "tech_type"])["end_value"]
    .sum()
    .groupby(["planning_year", "tech_type"])
    .agg(["min", "mean", "max"])
    .reset_index()
)
all_cap_range["relative_difference"] = (all_cap_range["max"] - all_cap_range["min"]) / all_cap_range["mean"]
all_cap_range

,planning_year,tech_type,min,mean,max,relative_difference
0,2027,Battery,29.0,29.000000,29.0,0.000000
1,2027,CCS,0.0,0.000000,0.0,NaN
2,2027,Hydrogen,1.0,1.000000,1.0,0.000000
3,2027,Natural Gas CC,21.0,1423.750000,4970.0,3.476032
4,2027,Natural Gas CT,13.0,13.000000,13.0,0.000000
5,2027,Nuclear,0.0,0.000000,0.0,NaN
6,2027,Solar,75.0,105.500000,136.0,0.578199
7,2027,Wind,115266.0,117741.500000,121794.0,0.055443
8,2030,Battery,600.0,600.000000,600.0,0.000000
9,2030,CCS,71961.0,74693.250000,77473.0,0.073795


In [13]:
all_cap_range.query("tech_type in ['Hydrogen', 'Geothermal']")

,planning_year,tech_type,min,mean,max,relative_difference
4,2027,Geothermal,676.0,713.250000,825.0,0.208903
6,2027,Hydrogen,1.0,1.000000,1.0,0.000000
16,2030,Geothermal,620.0,654.000000,754.0,0.204893
18,2030,Hydrogen,310.0,310.000000,310.0,0.000000
28,2035,Geothermal,591.0,622.500000,717.0,0.202410
30,2035,Hydrogen,0.0,56.666667,145.0,2.558824
40,2040,Geothermal,390.0,396.000000,398.0,0.020202
42,2040,Hydrogen,0.0,72.333333,167.0,2.308756
52,2045,Geothermal,217.0,219.500000,227.0,0.045558
54,2045,Hydrogen,0.0,82.333333,180.0,2.186235


In [32]:
all_cap_range.query("tech_type not in ['Geothermal', 'Distributed Solar','Hydrogen']").dropna()

,planning_year,tech_type,min,mean,max,relative_difference
0,2027,Battery,29.0,29.00,29.0,0.000000
3,2027,Natural Gas CC,21.0,1423.75,4970.0,3.476032
4,2027,Natural Gas CT,13.0,13.00,13.0,0.000000
6,2027,Solar,75.0,105.50,136.0,0.578199
7,2027,Wind,115266.0,117741.50,121794.0,0.055443
8,2030,Battery,600.0,600.00,600.0,0.000000
9,2030,CCS,71961.0,74693.25,77473.0,0.073795
11,2030,Natural Gas CC,1027.0,2490.50,4970.0,1.583216
12,2030,Natural Gas CT,168.0,168.00,168.0,0.000000
13,2030,Nuclear,69.0,69.00,69.0,0.000000


In [82]:
data = all_cap_range.query("planning_year in [2030, 2050] and relative_difference > 0").dropna()
data["mean"] /= 1000
# data["relative_difference"] = data["relative_difference"].replace(0, np.nan).dropna()
alt.Chart(
    # all_cap_range.query("tech_type not in ['Geothermal', 'Distributed Solar','Hydrogen']").dropna()
    data
    ).mark_point().encode(
    x=alt.X("mean").title("New Capacity (GW)"), # .scale(type="log"),
    y=alt.Y("relative_difference").scale(type="log").title("Relative Difference"),
    color=alt.Color("tech_type").title("Technology"),
    shape="tech_type",
    tooltip=["tech_type", "relative_difference", "mean", "planning_year"],
    
).properties(
    height=200,
    width=200
).facet(
    facet=alt.Facet("planning_year").title("Planning Year"),
    columns=2
)

alt.FacetChart(...)

In [38]:
all_tx = pd.read_csv(cwd / "all" / "annual_tx_expansion.csv")

In [44]:
all_tx.query("line_name == 'SRSG_to_TRE' and case == 'full-base-200'")

,model,line_name,planning_year,case,unit,value,end_value,line,start_region,dest_region
5434,GenX,SRSG_to_TRE,2023,full-base-200,MW,0.0,2.0,57.0,SRSG,TRE
5491,GenX,SRSG_to_TRE,2027,full-base-200,MW,2.0,NaN,57.0,SRSG,TRE
5548,GenX,SRSG_to_TRE,2030,full-base-200,MW,2453.0,NaN,57.0,SRSG,TRE
5605,GenX,SRSG_to_TRE,2035,full-base-200,MW,1003.0,NaN,57.0,SRSG,TRE
5662,GenX,SRSG_to_TRE,2040,full-base-200,MW,1085.0,NaN,57.0,SRSG,TRE
5719,GenX,SRSG_to_TRE,2045,full-base-200,MW,1297.0,NaN,57.0,SRSG,TRE
5776,GenX,SRSG_to_TRE,2050,full-base-200,MW,0.0,NaN,57.0,SRSG,TRE
5833,SWITCH,SRSG_to_TRE,2023,full-base-200,MW,0.0,0.0,NaN,SRSG,TRE
5890,SWITCH,SRSG_to_TRE,2027,full-base-200,MW,0.0,NaN,NaN,SRSG,TRE
5947,SWITCH,SRSG_to_TRE,2030,full-base-200,MW,1540.0,NaN,NaN,SRSG,TRE


In [54]:
base_total_tx_exp = (
    all_tx.query("case == 'full-base-200'")
    .groupby(["line_name", "start_region", "dest_region", "model"], as_index=False)[
        "value"
    ]
    .sum()
)
base_total_tx_exp

,line_name,start_region,dest_region,model,value
0,BASN_to_CANO,BASN,CANO,GenX,102.0
1,BASN_to_CANO,BASN,CANO,SWITCH,100.0
2,BASN_to_CANO,BASN,CANO,TEMOA,111.0
3,BASN_to_CANO,BASN,CANO,USENSYS,100.0
4,BASN_to_CASO,BASN,CASO,GenX,5920.0
...,...,...,...,...,...
240,TREW_to_TRE,TREW,TRE,TEMOA,11.0
241,TRE_to_TREW,TRE,TREW,GenX,16529.0
242,TRE_to_TREW,TRE,TREW,SWITCH,16529.0
243,TRE_to_TREW,TRE,TREW,TEMOA,16529.0


In [55]:
base_tx_exp_range = base_total_tx_exp.groupby(
    ["line_name", "start_region", "dest_region"], as_index=False
)["value"].agg(["min", "mean", "max"])
base_tx_exp_range["relative_difference"] = (
    base_tx_exp_range["max"] - base_tx_exp_range["min"]
) / base_tx_exp_range["mean"]
base_tx_exp_range

,line_name,start_region,dest_region,min,mean,max,relative_difference
0,BASN_to_CANO,BASN,CANO,100.0,103.25,111.0,0.106538
1,BASN_to_CASO,BASN,CASO,5920.0,5920.00,5920.0,0.000000
2,BASN_to_NWPP,BASN,NWPP,3350.0,3350.00,3350.0,0.000000
3,BASN_to_RMRG,BASN,RMRG,5518.0,5668.00,5835.0,0.055928
4,BASN_to_SRSG,BASN,SRSG,5385.0,5385.00,5385.0,0.000000
...,...,...,...,...,...,...,...
60,SRCE_to_SRSE,SRCE,SRSE,3196.0,3216.75,3275.0,0.024559
61,SRSG_to_BASN,SRSG,BASN,0.0,1.00,2.0,2.000000
62,SRSG_to_TRE,SRSG,TRE,4212.0,6251.50,8749.0,0.725746
63,TREW_to_TRE,TREW,TRE,0.0,4.00,11.0,2.750000


In [56]:
base_tx_exp_range.sort_values("relative_difference").tail(10)

,line_name,start_region,dest_region,min,mean,max,relative_difference
58,SRCA_to_SRCE,SRCA,SRCE,216.0,440.75,588.0,0.844016
32,MISW_to_SPPC,MISW,SPPC,100.0,253.50,542.0,1.743590
8,CASO_to_CANO,CASO,CANO,0.0,3.00,6.0,2.000000
7,CASO_to_BASN,CASO,BASN,0.0,1.00,2.0,2.000000
61,SRSG_to_BASN,SRSG,BASN,0.0,1.00,2.0,2.000000
34,NWPP_to_BASN,NWPP,BASN,0.0,1.00,2.0,2.000000
46,PJMW_to_MISE,PJMW,MISE,0.0,0.50,1.0,2.000000
63,TREW_to_TRE,TREW,TRE,0.0,4.00,11.0,2.750000
29,MISW_to_MISC,MISW,MISC,0.0,0.00,0.0,NaN
35,NWPP_to_CASO,NWPP,CASO,0.0,0.00,0.0,NaN


In [60]:
base_tx_exp_range.sort_values("mean")

,line_name,start_region,dest_region,min,mean,max,relative_difference,lat1,lon1,lat2,lon2
35,NWPP_to_CASO,NWPP,CASO,0.0,0.00,0.0,NaN,45.925635,-117.047147,35.083692,-117.407157
29,MISW_to_MISC,MISW,MISC,0.0,0.00,0.0,NaN,45.702637,-95.996143,39.075295,-89.537578
46,PJMW_to_MISE,PJMW,MISE,0.0,0.50,1.0,2.000000,39.422340,-81.908171,43.569687,-84.579049
61,SRSG_to_BASN,SRSG,BASN,0.0,1.00,2.0,2.000000,34.357403,-109.333542,40.570631,-114.459232
34,NWPP_to_BASN,NWPP,BASN,0.0,1.00,2.0,2.000000,45.925635,-117.047147,40.570631,-114.459232
...,...,...,...,...,...,...,...,...,...,...,...
18,MISC_to_PJMW,MISC,PJMW,10927.0,12454.25,14071.0,0.252444,39.075295,-89.537578,39.422340,-81.908171
55,SPPS_to_SRSG,SPPS,SRSG,10101.0,14181.25,16181.0,0.428735,34.990645,-98.457473,34.357403,-109.333542
64,TRE_to_TREW,TRE,TREW,16529.0,16529.00,16529.0,0.000000,30.318114,-97.544556,32.161745,-101.589682
26,MISS_to_SPPS,MISS,SPPS,16970.0,22965.50,26104.0,0.397727,32.374053,-91.562627,34.990645,-98.457473


In [52]:
gdf = gpd.read_file("conus_26z_latlon_simple.geojson")
gdf = gdf.rename(columns={"model_region": "zone"})

In [64]:
geoshape_kwargs = {
        "stroke": "white",
        "fill": "silver",
    }
gdf["lat"] = gdf.geometry.centroid.y
gdf["lon"] = gdf.geometry.centroid.x
base_tx_exp_range["lat1"] = base_tx_exp_range["start_region"].map(gdf.set_index("zone")["lat"])
base_tx_exp_range["lon1"] = base_tx_exp_range["start_region"].map(gdf.set_index("zone")["lon"])
base_tx_exp_range["lat2"] = base_tx_exp_range["dest_region"].map(gdf.set_index("zone")["lat"])
base_tx_exp_range["lon2"] = base_tx_exp_range["dest_region"].map(gdf.set_index("zone")["lon"])

background = (
            alt.Chart(gdf)
            .mark_geoshape(**geoshape_kwargs)
            .project(type="albersUsa")
            .properties(height=325, width=400)
        )
lines = alt.Chart(
    base_tx_exp_range.query("mean > 500")
    ).mark_rule().encode(
    latitude="lat1",
    longitude="lon1",
    latitude2="lat2",
    longitude2="lon2",
    strokeWidth=alt.StrokeWidth("mean").title("Average Expansion (MW)"), #.scale(type="log"),
    color=alt.Color("relative_difference:Q")
    .scale(scheme="magma", reverse=True)
    .title("Model Difference"),
    tooltip=["line_name", "min", "mean", "max", "relative_difference"]
).project(type="albersUsa")

background + lines

/var/folders/nr/y3csklpd0l77v_k39y46s2rc0000gn/T/ipykernel_61113/884558716.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["lat"] = gdf.geometry.centroid.y
/var/folders/nr/y3csklpd0l77v_k39y46s2rc0000gn/T/ipykernel_61113/884558716.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["lon"] = gdf.geometry.centroid.x


alt.LayerChart(...)

In [65]:
geoshape_kwargs = {
        "stroke": "white",
        "fill": "silver",
    }
gdf["lat"] = gdf.geometry.centroid.y
gdf["lon"] = gdf.geometry.centroid.x
base_tx_exp_range["lat1"] = base_tx_exp_range["start_region"].map(gdf.set_index("zone")["lat"])
base_tx_exp_range["lon1"] = base_tx_exp_range["start_region"].map(gdf.set_index("zone")["lon"])
base_tx_exp_range["lat2"] = base_tx_exp_range["dest_region"].map(gdf.set_index("zone")["lat"])
base_tx_exp_range["lon2"] = base_tx_exp_range["dest_region"].map(gdf.set_index("zone")["lon"])

background = (
            alt.Chart(gdf)
            .mark_geoshape(**geoshape_kwargs)
            .project(type="albersUsa")
            .properties(height=325, width=400)
        )
lines = alt.Chart(
    base_tx_exp_range.query("mean > 500")
    ).mark_rule().encode(
    latitude="lat1",
    longitude="lon1",
    latitude2="lat2",
    longitude2="lon2",
    strokeWidth=alt.StrokeWidth("mean").title("Average Expansion (MW)").scale(type="log"),
    color=alt.Color("relative_difference:Q")
    .scale(scheme="magma", reverse=True)
    .title("Model Difference"),
    tooltip=["line_name", "min", "mean", "max", "relative_difference"]
).project(type="albersUsa")

background + lines

/var/folders/nr/y3csklpd0l77v_k39y46s2rc0000gn/T/ipykernel_61113/4081479056.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["lat"] = gdf.geometry.centroid.y
/var/folders/nr/y3csklpd0l77v_k39y46s2rc0000gn/T/ipykernel_61113/4081479056.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["lon"] = gdf.geometry.centroid.x


alt.LayerChart(...)

In [68]:
alt.Chart(
    base_tx_exp_range.query("mean > 500")
    ).mark_point().encode(
    x=alt.X("mean").title("Capacity (MW)"), #.scale(type="log"),
    y=alt.Y("relative_difference"), #.scale(type="log"),
    # color="tech_type",
    # shape="tech_type",
    # tooltip=["tech_type", "relative_difference", "mean", "planning_year"],
    
).properties(
    height=200,
    width=400
)

alt.Chart(...)

In [ ]:
alt.Chart()